In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Reload config without restarting the kernel
import importlib
import config
importlib.reload(config)
from config import facemesh_included, DATA_PATH, actions, no_sequences, sequence_length

# Define the path where keypoints are stored
AUGMENTED_KEYPOINTS_PATH = "augmentation_keypoints"

# Validate that config values are correctly imported
print(f"Facemesh included: {facemesh_included}")
# print(f"Data path: {DATA_PATH}")
print(f"Actions: {actions}")
print(f"No sequences: {no_sequences}")
print(f"Sequence length: {sequence_length}")

# Create a label map
label_map = {label: num for num, label in enumerate(actions)}

sequences, labels = [], []

# Loop through each action to load keypoints
for action in actions:
    action_path = os.path.join(AUGMENTED_KEYPOINTS_PATH, action)
    if not os.path.exists(action_path):
        print(f"Action path does not exist: {action_path}")
        continue
    
    for sequence in os.listdir(action_path):
        sequence_path = os.path.join(action_path, sequence)
        if not os.path.isdir(sequence_path):
            print(f"Sequence path is not a directory: {sequence_path}")
            continue

        # Check if there are augmentation directories within the sequence directory
        augmentations_present = any(os.path.isdir(os.path.join(sequence_path, aug)) for aug in os.listdir(sequence_path))
        if not augmentations_present:
            print(f"No augmentation directories found in: {sequence_path}")
            continue

        for augmentation in os.listdir(sequence_path):
            augmentation_path = os.path.join(sequence_path, augmentation)
            if not os.path.isdir(augmentation_path):
                print(f"Augmentation path is not a directory: {augmentation_path}")
                continue

            window = []
            valid_sequence = True  # Assume the sequence is valid unless proven otherwise

            for frame_num in range(sequence_length):
                frame_path = os.path.join(augmentation_path, f"{frame_num}.npy")
                if not os.path.exists(frame_path):
                    print(f"Frame path does not exist: {frame_path}")
                    valid_sequence = False
                    break

                # Load the frame
                res = np.load(frame_path)
                # Validate the length of the array
                if not facemesh_included:
                    res = res[:258]
                window.append(res)
            
            if valid_sequence:
                sequences.append(window)
                labels.append(label_map[action])
            else:
                print(f"Invalid sequence for action: {action}, sequence: {sequence}, augmentation: {augmentation}")

# Convert to numpy arrays
X = np.array(sequences)
y = to_categorical(labels).astype(int)

# Save the processed data
output_data_path = os.path.join(AUGMENTED_KEYPOINTS_PATH) #, 'processed_data')
os.makedirs(output_data_path, exist_ok=True)
np.save(os.path.join(output_data_path, 'X_augmented'), X)
np.save(os.path.join(output_data_path, 'y_augmented'), y)

print("Data pre-processing complete.")


Facemesh included: False
Actions: ['background', 'NoHands', 'ILoveYou', 'Yes', 'No', 'Hi', 'ThankYou', 'Me', 'You', 'It', 'Feel', 'Happy', 'Hungry', 'Eat', 'Bread', 'Chocolate', 'Tired']
No sequences: 30
Sequence length: 30
Data pre-processing complete.
